<a href="https://colab.research.google.com/github/JPleal01/Dio_IA/blob/main/YOLO_DIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/pjreddie/darknet
%cd darknet
!make

In [ ]:
!wget https://data.pjreddie.com/files/yolov3.weights


In [ ]:
!./darknet detect cfg/yolov3.cfg yolov3.weights data/dog.jpg

In [ ]:
#Baixar arquivos
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")

  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()


def upload():
  from google.colab import files
  uploaded = files.upload()
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)
def download(path):
  from google.colab import files
  files.download(path)




In [ ]:
imShow('predictions.jpg')

In [ ]:
import os

# Criar pastas
os.makedirs('./coco/images', exist_ok=True)
os.makedirs('./coco/annotations', exist_ok=True)

# Baixar imagens de treino da COCO 2017
!wget http://images.cocodataset.org/zips/train2017.zip
!unzip -q train2017.zip -d ./coco/images/

# Baixar anotações
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip -q annotations_trainval2017.zip -d ./coco/annotations/

In [ ]:
import json

# Carregar anotações
with open('./coco/annotations/annotations/instances_train2017.json') as f:
    data = json.load(f)

# IDs das classes desejadas
classes_desejadas = ['person', 'dog']
ids_desejados = [cat['id'] for cat in data['categories'] if cat['name'] in classes_desejadas]

# Filtrar imagens
imagens_filtradas = set()
for ann in data['annotations']:
    if ann['category_id'] in ids_desejados:
        imagens_filtradas.add(ann['image_id'])

print(f"Total de imagens com classes desejadas: {len(imagens_filtradas)}")

In [ ]:
!pip install pycocotools tqdm


import os
import json
from tqdm import tqdm
from PIL import Image

# Caminhos
coco_json = './coco/annotations/annotations/instances_train2017.json'
images_dir = './coco/images/train2017'
labels_dir = './coco/labels/train2017'
os.makedirs(labels_dir, exist_ok=True)

# Carregar anotações COCO
with open(coco_json) as f:
    coco = json.load(f)

# Mapear classes desejadas
classes_desejadas = ['person', 'dog']
class_map = {cat['id']: i for i, cat in enumerate(coco['categories']) if cat['name'] in classes_desejadas}
name_to_id = {cat['name']: cat['id'] for cat in coco['categories']}

# Mapear imagens
image_id_to_filename = {img['id']: img['file_name'] for img in coco['images']}
image_id_to_size = {img['id']: (img['width'], img['height']) for img in coco['images']}

# Agrupar anotações por imagem
annotations_by_image = {}
for ann in coco['annotations']:
    if ann['category_id'] in class_map:
        img_id = ann['image_id']
        annotations_by_image.setdefault(img_id, []).append(ann)

# Converter para YOLO
for img_id, anns in tqdm(annotations_by_image.items()):
    img_file = image_id_to_filename[img_id]
    width, height = image_id_to_size[img_id]
    label_path = os.path.join(labels_dir, img_file.replace('.jpg', '.txt'))

    with open(label_path, 'w') as f:
        for ann in anns:
            x, y, w, h = ann['bbox']
            x_center = (x + w / 2) / width
            y_center = (y + h / 2) / height
            w /= width
            h /= height
            class_id = class_map[ann['category_id']]
            f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")


In [ ]:
!./darknet detector train cfg/coco.data cfg/yolov3.cfg darknet53.conv.74
